In [1]:
import numpy as np
import matplotlib.pyplot as plt

import os
import PIL

import tensorflow as tf
from tensorflow import keras

In [2]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url,  cache_dir='.', untar=True)
# cache_dir indicates where to download data. I specified . which means current directory
# untar true will unzip it

228827136/228813984 [==============================] - 1s 0us/step


In [3]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [4]:
flowers_images_dict = {
    'roses': list(data_dir.glob('roses/*')),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*'))
}

In [5]:
flowers_labels_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4
}

In [6]:
import cv2

X, y = [], []

for flower_name, images in flowers_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img ,(224,224))
        X.append(resized_img)
        y.append(flowers_labels_dict[flower_name])

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [9]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# CNN  MODEL


In [10]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 33s 26ms/step - loss: 2.4939 - accuracy: 0.3103
Epoch 2/10
86/86 [==============================] - 2s 25ms/step - loss: 1.1455 - accuracy: 0.5483
Epoch 3/10
86/86 [==============================] - 2s 26ms/step - loss: 0.8208 - accuracy: 0.6915
Epoch 4/10
86/86 [==============================] - 2s 26ms/step - loss: 0.4874 - accuracy: 0.8325
Epoch 5/10
86/86 [==============================] - 2s 25ms/step - loss: 0.2320 - accuracy: 0.9273
Epoch 6/10
86/86 [==============================] - 2s 26ms/step - loss: 0.1265 - accuracy: 0.9669
Epoch 7/10
86/86 [==============================] - 2s 26ms/step - loss: 0.0597 - accuracy: 0.9880
Epoch 8/10
86/86 [==============================] - 2s 25ms/step - loss: 0.0299 - accuracy: 0.9938
Epoch 9/10
86/86 [==============================] - 2s 26ms/step - loss: 0.0553 - accuracy: 0.9876
Epoch 10/10
86/86 [==============================] - 2s 26ms/step - loss: 0.0138 - accuracy: 0.9989


In [11]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 1s 20ms/step - loss: 2.4136 - accuracy: 0.5708


[2.4136433601379395, 0.570806086063385]

Training accuracy is 0.99 whereas testing accuracy is 0.57 which is due to overfitting of the model. This can be prevented by data augmentation.

# Data augmentation

In [12]:
# data augmentation layer
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(224, 
                                                              224,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [13]:

model = keras.Sequential([
    data_augmentation,
    keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(5, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 4s 32ms/step - loss: 1.7281 - accuracy: 0.2892
Epoch 2/10
86/86 [==============================] - 3s 32ms/step - loss: 1.3189 - accuracy: 0.4437
Epoch 3/10
86/86 [==============================] - 3s 32ms/step - loss: 1.1253 - accuracy: 0.5680
Epoch 4/10
86/86 [==============================] - 3s 31ms/step - loss: 0.9924 - accuracy: 0.6214
Epoch 5/10
86/86 [==============================] - 3s 32ms/step - loss: 0.9254 - accuracy: 0.6446
Epoch 6/10
86/86 [==============================] - 3s 31ms/step - loss: 0.8737 - accuracy: 0.6562
Epoch 7/10
86/86 [==============================] - 3s 31ms/step - loss: 0.8433 - accuracy: 0.6755
Epoch 8/10
86/86 [==============================] - 3s 32ms/step - loss: 0.8164 - accuracy: 0.6893
Epoch 9/10
86/86 [==============================] - 3s 32ms/step - loss: 0.7796 - accuracy: 0.6969
Epoch 10/10
86/86 [==============================] - 3s 31ms/step - loss: 0.7513 - accuracy: 0.7089


In [14]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 1s 19ms/step - loss: 0.7767 - accuracy: 0.7157


[0.7766925096511841, 0.7156862616539001]

Definitely the result is better with data augmentation. But is there any other way to increase it further? YES. 

# Transfer Learning

In [15]:
import tensorflow_hub as hub

feature_extractor_model = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=(224, 224, 3), trainable=False)

In [16]:
num_of_flowers = 5

model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [17]:
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

model.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
86/86 [==============================] - 7s 40ms/step - loss: 0.7991 - acc: 0.6969
Epoch 2/10
86/86 [==============================] - 3s 40ms/step - loss: 0.4164 - acc: 0.8557
Epoch 3/10
86/86 [==============================] - 3s 39ms/step - loss: 0.3279 - acc: 0.8884
Epoch 4/10
86/86 [==============================] - 3s 39ms/step - loss: 0.2708 - acc: 0.9142
Epoch 5/10
86/86 [==============================] - 3s 40ms/step - loss: 0.2364 - acc: 0.9295
Epoch 6/10
86/86 [==============================] - 3s 39ms/step - loss: 0.2083 - acc: 0.9430
Epoch 7/10
86/86 [==============================] - 3s 40ms/step - loss: 0.1859 - acc: 0.9513
Epoch 8/10
86/86 [==============================] - 3s 39ms/step - loss: 0.1676 - acc: 0.9582
Epoch 9/10
86/86 [==============================] - 3s 40ms/step - loss: 0.1468 - acc: 0.9669
Epoch 10/10
86/86 [==============================] - 3s 40ms/step - loss: 0.1352 - acc: 0.9702


In [18]:
model.evaluate(X_test_scaled,y_test)

29/29 [==============================] - 2s 53ms/step - loss: 0.3494 - acc: 0.8802


[0.34936460852622986, 0.8801742792129517]

0.97 training accuracy and 0.88 testing accuracy defenetively an amazing result